# Make Sure You Are Ready to Go

$\renewcommand{\ket}[1]{\left| #1 \right\rangle}
\renewcommand{\bra}[1]{\left\langle #1 \right|}
\renewcommand{\braket}[2]{\left\langle #1 | #2 \right\rangle}
\newcommand{\ketbra}[2]{\left| #1 \right\rangle\!\left\langle #2 \right|}$

If you haven't done it yet, try running the following lines of code and use the [registration and installation](https://docs.classiq.io/latest/classiq_101/registration_installations/) page if you are having difficulty setting up your environment.\
Uncomment and run the following command to install or update to the latest version of the Classiq SDK (if not installed yet):


In [1]:
# pip install -U classiq

Note: you may need to restart the kernel to use updated packages.


In [2]:
import classiq

Uncomment and run the following command if your machine has not been authenticated yet, you only need to run it once!


In [3]:
# classiq.authenticate()

Now you are good to go!


# Rydberg Phase Diagram


Before starting to code, let us reiterating some theory on Rydberg atoms - the subject of this challenge. They interact via the following Hamiltonian:

$$
H = \frac{\Omega}{2} \sum_{i=1}^N X_i
    - \delta \sum_{i=1}^N n_i
    + \sum_{i \lt j} \frac{\Omega R_b^6 }{(a|i-j|)^6} n_i n_j.
$$

You can find the phase diagram for a $51$-atom chain below. It is obtained by fixing $a=1$ and $\Omega=1$ and varying $\delta$ and $R_b$.


<img src="phase_diagram.png" alt="Phase Diagram" width="800">


Fig.1: Phase diagram of the 1D Rydberg Hamiltonian, traced out by (left) bipartite entanglement entropy and (right) expectation value of the number of Rydberg excitations. Plots are obtained using tensor-network representation of the ground states of $H$.


In this challenge, we focus on distinguishing between the $Z2$ phase, where the ground state of $H$ has large overlap with the state $\ket{rgr\ldots gr}$, and the $Z3$ phase, where the ground state overlaps strongly with basis states of the form $\ket{\ldots rggrgg\ldots}$.

Evidently, such systems can be efficiently studied using tensor networks. However, this challenge prepares us for a more realistic scenario in which we only have access to measurement outcomes from the ground state of some Hamiltonian, and our goal is to determine which phase of matter the state belongs to.


# Loading and Processing Measurement Data

Training data for your model contains measurement results in randomized bases performed on a 51-qubit Rydberg atoms chain. We load training data from the .npz file in the next cell.


In [4]:
import numpy as np

In [5]:
loaded = np.load("training_data.npz", allow_pickle=True)

unprocessed_features = loaded["features"].tolist()
unprocessed_labels = loaded["labels"].tolist()

print(f'There are {len(unprocessed_features)} data points')
print(f'There were T = {len(unprocessed_features[0])} measurements performed for each data point')
print(f'The measurements were performed on {len(unprocessed_features[0][0])} qubits')
print(f'Example: 2nd experiment result of 8th data point -> {unprocessed_features[7][1]}')
print(f'Example: label for the 8th data point -> {unprocessed_labels[7]}')
print(f'All labels: {unprocessed_labels}')

There are 20 data points
There were T = 500 measurements performed for each data point
The measurements were performed on 51 qubits
Example: 2nd experiment result of 8th data point -> ['r', '-', 'i', 'i', 'r', '+', 'r', 'g', '+', '-i', 'r', '-', '-', 'g', '-i', '-', 'r', 'g', 'i', 'i', 'i', '-', 'r', '-i', 'r', 'i', 'r', '+', 'i', 'g', '-', '-i', '-', 'g', '-i', 'i', 'r', '-', '-', '-', 'i', 'i', '-i', 'g', '-i', 'g', 'r', '-', 'r', '+', '-']
Example: label for the 8th data point -> Z2
All labels: ['Z2', 'Z2', 'Z2', 'Z2', 'Z2', 'Z2', 'Z2', 'Z2', 'Z2', 'Z2', 'Z3', 'Z3', 'Z3', 'Z3', 'Z3', 'Z3', 'Z3', 'Z3', 'Z3', 'Z3']


In the above,

-   $\ket{g}$ is the atomic ground state, which is a $+1$-eigenstate of Pauli $Z$
-   $\ket{r}$ is the highly excited Rydberg state, which is a $-1$-eigenstate of Pauli $Z$
-   $\ket{+} = \frac{1}{\sqrt2}(\ket{g} + \ket{r})$, a $+1$-eigenstate of Pauli $X$
-   $\ket{-} = \frac{1}{\sqrt2}(\ket{g} - \ket{r})$, a $-1$-eigenstate of Pauli $X$
-   $\ket{+i} = \frac{1}{\sqrt2}(\ket{g} +i\ket{r})$, a $+1$-eigenstate of Pauli $Y$
-   $\ket{-i} = \frac{1}{\sqrt2}(\ket{g} -i \ket{r})$, a $-1$-eigenstate of Pauli $Y$.

It is up to you how to convert the features into classical shadows and labels into numbers and then both into training data for your model. For example, you could assign $-1$ to $Z2$ and $+1$ to $Z3$.

**Note:** If you decide to define any helper classes/functions in a separate Python file, please submit it alongside your solution notebook, so we can run and grade it properly


In [6]:
### Your Code Goes Here: ###
# Using mapping:
# Z2 -> -1
# Z3 -> +1
labels = [1 if (x == 'Z3') else -1 for x in unprocessed_labels]
print(f'Processed labels: {labels}')

raw_features = []

for data_point in unprocessed_features:
    processed_data_point = []

    for measurement_round in data_point:
        processed_measurements = []

        for qubit_outcome in measurement_round:
            if qubit_outcome[0] == 'g':
                value = 1
            elif qubit_outcome[0] == 'r':
                value = -1
            else:
                value = 0

            processed_measurements.append(value)

        processed_data_point.append(processed_measurements)
    
    raw_features.append(processed_data_point)

print(f'Example: 2nd experiment result of 8th data point -> {raw_features[7][1]}')

Processed labels: [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Example: 2nd experiment result of 8th data point -> [-1, 0, 0, 0, -1, 0, -1, 1, 0, 0, -1, 0, 0, 1, 0, 0, -1, 1, 0, 0, 0, 0, -1, 0, -1, 0, -1, 0, 0, 1, 0, 0, 0, 1, 0, 0, -1, 0, 0, 0, 0, 0, 0, 1, 0, 1, -1, 0, -1, 0, 0]


As the terms $Z_2$-ordered and $Z_3$-ordered suggest, and as we are told in the provided references in the challenge statement, the ground states (whose measurements we are provided) strongly overlap with:

-   $\ket{10101010...}$ or $\ket{01010101...}$ for $Z_2$-ordered phase,
-   or with $\ket{1001001...}, \ket{01001001...},$ or $\ket{00100100...}$ for $Z_3$-ordered phase.

We can see from the pattern it is sufficient to look at just the first four qubits to distinguish between these patterns. I.e., we are distingiushing between

-   $\ket{1010...}$ or $\ket{0101...}$ for $Z_2$-ordered phase,
-   or with $\ket{1001...}, \ket{0100...},$ or $\ket{0010...}$ for $Z_3$-ordered phase.


In [7]:
raw_features = np.array(raw_features)

# Consists of <Z_i> expectation values, averaged over all T=500 experiments (measurement rounds) for that data point
flattened = np.mean(raw_features, axis=1)

rounded = np.where(flattened >= 0, 1, -1)
mapped = np.where(rounded == 1, 0, 1)

# We will only use the first 4 qubits. There is enough information in the first 4 qubits to classify the data points.
features = mapped[:, :4]

# Defining a Quantum Model

In this section, you will create a QML model for classifying the quantum phases. This will include 3 stages:

-   First, you will need to decide on the data encoding scheme, i.e. loading numerical features you obtained above into the quantum circuit.
-   Then, you will need to come up with an ansatz - a parametrized quantum circuit, which will be optimized to perform classification.
-   Finally, to readout classical information from the quantum model, you will need to perform some sort of measurement on the resultant quantum state. Perhaps, you could extract an expectation value of some Pauli-string $P \in \{I, X, Y, Z\}^{\otimes N}$, so that $\langle P \rangle < b$ is interpreted as $Z2$ and $\langle P \rangle > b$ is interpreted as $Z3$ for some decision boundary $b$.

There are several approaches to QML in Classiq, linked below.

You may find the following guides useful:

-   QML with Classiq: http://docs.classiq.io/latest/user-guide/read/qml_with_classiq_guide/
-   Variational Model Example: https://github.com/Classiq/classiq-library/blob/main/algorithms/qaoa/maxcut/qaoa_max_cut.ipynb
-   Hybrid QNN: https://docs.classiq.io/latest/explore/algorithms/qml/hybrid_qnn/hybrid_qnn_for_subset_majority/

Although the 2nd guide describes a hybrid model, **you may not implement a hybrid model**, the guide should only be used as a reference as to how to implement QML.

**Warning**: Training using the Classiq PyTorch integration may take a prohibitive amount of time. Consider this when choosing an approach.


In [8]:
from classiq import *
from classiq.execution import *
# You might need to make additional imports depending on your implementation

In [9]:
# The following function signatures are only a template to get you started. You may, and should change them to suit your needs!

#Change these according to your implementation.

num_qubits = 5
num_auxillary_qubits = 1
num_data_qubits = num_qubits - num_auxillary_qubits 

num_weights = num_data_qubits - 2 # Skipping parametrising first two qubits. See ansatz definition.
feature_length = features.shape[1]

@qfunc
def encoding(feature: CArray[CReal, feature_length], wires: QArray) -> None: # your encoding scheme
    ### Your Code Goes Here: ###
    for i in range(num_qubits - 1):
        RX(np.pi * feature[i], wires[i])

@qfunc
def ansatz(weights: CArray[CReal, num_weights], wires: QArray) -> None: # your ansatz
    ### Your Code Goes Here: ###
    for i in range(num_data_qubits - 2):
        # First qubits of the Reydberg chain have strongest magnitude expectation values in the true ground state. We skip optimising these to reduce parameter count 
        wire_index = i + 2

        RX(2 * np.pi * weights[i], wires[wire_index])

@qfunc
def main(
            feature: CArray[CReal, feature_length],
            weights: CArray[CReal, num_weights],
            result: Output[QArray[QBit]],
        ) -> None:
    '''
    Combine the encoding scheme and parametrized quantum circuit you came up with
    to encode and then process data
    '''
    allocate(num_qubits, result)
    ### Your Code Goes Here: ###
    encoding(feature, result)
    ansatz(weights, result)

    for i in range(0, num_qubits - 3):
        CCX([result[i], result[i + 2]], result[num_qubits-1])


### Synthesis

Before training, you must synthesize your model into a quantum program. Placeholders for your parameters will be automatically generated.

You may find the following documentation useful: https://docs.classiq.io/latest/sdk-reference/synthesis/


In [10]:
# Set your preferences here!

NUM_SHOTS = None

# SIMULATOR is the default backend.
BACKEND_PREFERENCES = ClassiqBackendPreferences(
        backend_name=ClassiqSimulatorBackendNames.SIMULATOR
)

# Vary this to achieve the best optimization of your circuit.
OPTIMIZATION_PARAMETER = 'no_opt' 

QMOD = create_model(
            main,
            execution_preferences=ExecutionPreferences(
                num_shots=NUM_SHOTS, backend_preferences=BACKEND_PREFERENCES
            ),
            constraints=Constraints(optimization_parameter=OPTIMIZATION_PARAMETER)
        )

QPROG = synthesize(QMOD)

# Use the show() function to display your circuit in the browser.
show(QPROG)

Quantum program link: https://platform.classiq.io/circuit/2xGJMY9RGfARkhr3bYFa3aeyNqM?login=True&version=0.79.1


# Training the Model

Here, you will optimize the weights in ansatz, so that the model can distiguish between the phases.

You can find the following Classiq tutorial and documentation useful:

-   Execution: https://docs.classiq.io/latest/sdk-reference/execution/
-   Execution Session: https://docs.classiq.io/latest/user-guide/execution/ExecutionSession/
-   Executing With Parameters: https://docs.classiq.io/latest/qmod-reference/language-reference/quantum-entry-point/

It is highly recommended to use an ExecutionSession if you are executing the same circuit with different parameters many times. It is not needed to train parameters using the Classiq PyTorch integration.

If you are not using the PyTorch integration, you will need an objective (also known as a 'loss', or 'cost') function. Depending on your implementation, you will need to either minimize or maximize it in training.


In [11]:
### Your Code Goes Here: ###
from scipy.optimize import minimize

input_features = features.tolist()

# Measure auxillary qubit
hamiltonian = [PauliTerm([Pauli.Z, Pauli.I, Pauli.I, Pauli.I, Pauli.I], coefficient=1),]

def execute_circuit_and_get_expectation_value(features, weights):
    batch_params = [{'feature': features[i], 'weights': weights} for i in range(len(features))]

    # results = es.batch_sample(batch_params)
    estimates = es.batch_estimate(hamiltonian, batch_params)

    # Extra real part of expectation values
    return [estimate.value.real for estimate in estimates]

def loss_fn(weights):
    weights = weights.tolist() if isinstance(weights, np.ndarray) else weights
    exp_vals = execute_circuit_and_get_expectation_value(input_features, weights)
    target_vals = labels 
    return mean_squared_error(exp_vals, target_vals)

def mean_squared_error(x, y):
    return sum((a - b) ** 2 for a, b in zip(x, y)) / len(x)


init_weights = (np.random.randn(num_weights) * 0.1).tolist()

# Train model
with ExecutionSession(QPROG) as es:
    result = minimize(loss_fn, init_weights, method="COBYLA", options={"maxiter": 1000, "tol": 1e-3})

In [12]:
# Extract optimised model weights and loss
optimised_weights = result.x.tolist()
loss = result.fun

print("Optimised weights:", optimised_weights)
print("Final loss (Mean-Squared Error):", loss)

Optimised weights: [0.9980758490115569, -0.11192550299445024]
Final loss (Mean-Squared Error): 9.5367431640625e-08


Training that takes too long may make it impossible to grade your submission.


# Testing the Model

Good job! Now it's time to see whether the model you designed can successfully perform the classification. For this, compare the predictions of your model to the actual labels.

If the model does not perform well, try modifying the encoding and/or the ansatz (by using different number of parameters/qubits/ansatz layers/...)


In [13]:
### Your Code Goes Here: ###

# Execute circuit with optimised weights
with ExecutionSession(QPROG) as es:
    exp_vals = execute_circuit_and_get_expectation_value(input_features, optimised_weights)

# Map circuit outputs to predictions
predictions = [1 if exp_val >= 0 else -1 for exp_val in exp_vals]

# Calculate accuracy
num_correct = sum(1 for prediction, true_label in zip(predictions, labels) if prediction == true_label)
accuracy = num_correct / len(predictions)

print(f'Accuracy:\t{round(accuracy * 100, 2)}%')
print(f'Predicted:\t{predictions}')
print(f'True labels:\t{labels}')

NameError: name 'processed_labels' is not defined

## Grading

You will be evaluated on the accuracy, depth, width of your model and the number of parameters in your model.


The following function will return the width and depth of your model as they will be used in grading. Use it to self-evaluate your model.


In [ ]:
from classiq import QuantumProgram

def get_metrics(qprog):
    """
    Extract circuit metrics from a quantum program.

    Parameters:
        qprog: The quantum program object.

    Returns:
        dict: A dictionary containing the circuit metrics:
              - "depth": Circuit depth
              - "width": Circuit width
    """
    circuit = QuantumProgram.from_qprog(qprog)

    metrics = {
        "depth": circuit.transpiled_circuit.depth,
        "width": circuit.data.width,
    }

    return metrics

def calculate_score(accuracy, metrics):
    A = accuracy
    P = num_weights
    D = metrics['depth']
    W = num_qubits

    score = A - 0.1 * P - 0.0002 * D - 0.1 * W
    return round(score, 5)

In [ ]:
metrics = get_metrics(QPROG)

print(metrics)
print(f'Score: {calculate_score(accuracy, metrics)}')
print(f'Max attainable score with circuit design: {calculate_score(1, metrics)}')

{'depth': 20, 'width': 5}
Score: 0.296
Max attainable score with circuit design: 0.296


# Submission

You will submit this notebook, your trained parameters, and your quantum model.


In [ ]:
# Do not change this cell

import os

def save_qprog(qprog, team_name: str, folder="."):
    assert isinstance(team_name, str)
    file_name = f"{team_name.replace(' ','_')}.qprog"
    with open(os.path.join(folder, file_name), 'w') as f:
        f.write(qprog.model_dump_json(indent=4))
        
def save_params(params, team_name: str, folder="."):
    assert isinstance(team_name, str)
    file_name = f"{team_name.replace(' ','_')}.npz"
    with open(os.path.join(folder, file_name), 'wb') as f:
        np.savez(f, params=params)

In [ ]:
# Change to your team name!!

TEAM_NAME = "NotJustAPhase"

# Insert your trained parameters here!

TRAINED_PARAMS = optimised_weights

save_qprog(QPROG, team_name=TEAM_NAME)
save_params(params=TRAINED_PARAMS, team_name=TEAM_NAME)
